In [67]:
import requests
import pandas as pd

api_key = 'ITL1B9MC4L5G7SEN'
symbol = 'SIE.DE'
output_size = 'full'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={output_size}&apikey={api_key}'
r = requests.get(url)
time_series = r.json()['Time Series (Daily)']

data = []
for date, values in time_series.items():
    data.append([
        date,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        int(values["5. volume"])
    ])

df0 = pd.DataFrame(data, columns=["date", "open", "high", "low", "close", "volume"])
print(df0.shape)
df0.head(15)

(4943, 6)


,date,open,high,low,close,volume
0,2024-06-14,170.80,171.10,165.68,165.74,1686923
1,2024-06-13,177.28,177.56,170.44,170.84,1395794
2,2024-06-12,173.12,178.58,172.30,178.10,975006
3,2024-06-11,174.70,175.44,172.18,172.46,953042
4,2024-06-10,174.12,174.52,172.06,174.44,717411
5,2024-06-07,175.00,175.26,172.62,174.44,1034765
6,2024-06-06,178.88,179.52,175.14,175.68,826573
7,2024-06-05,175.60,178.82,175.38,178.82,828300
8,2024-06-04,178.58,179.00,174.28,174.28,1232510
9,2024-06-03,178.80,179.60,177.10,178.16,745486


In [68]:
import pandas_ta as pta
import ta

df = df0.copy()

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df['RSI'] = pta.rsi(df['close'], length=11)
df['William%R'] = pta.willr(df['high'], df['low'], df['close'], length=11)
df['WMA'] = pta.wma(df['close'], length=11)
df['EMA'] = pta.ema(df['close'], length=11)
df['SMA'] = pta.sma(df['close'], length=11)
df['HMA'] = pta.hma(df['close'], length=11)
df['CCI'] = pta.cci(df['high'], df['low'], df['close'], length=11)
df['CMO'] = pta.cmo(df['close'], length=11)

macd = pta.macd(df['close'])
df['MACD'] = macd.iloc[:, 0] 
df['MACD_signal'] = macd.iloc[:, 1] 

ppo = pta.ppo(df['close'])
df['PPO'] = ppo.iloc[:, 0] 
df['PPO_signal'] = ppo.iloc[:, 1]

df['ROC'] = pta.roc(df['close'], length=11)
df['CMFI'] = ta.volume.ChaikinMoneyFlowIndicator(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], window=11).chaikin_money_flow()

dmi = pta.adx(df['high'], df['low'], df['close'], length=11)
df['DMI'] = dmi['ADX_11']
df['DI+'] = dmi['DMP_11']
df['DI-'] = dmi['DMN_11']

df['PSI'] = pta.pvt(df['close'], df['volume'])
df['TripleEMA'] = pta.ema(df['close'], length=11)

df = df.dropna()
print(df.info())
df.head(15)

,open,high,low,close,volume,RSI,William%R,WMA,EMA,SMA,...,MACD_signal,PPO,PPO_signal,ROC,CMFI,DMI,DI+,DI-,PSI,TripleEMA
date,,,,,,,,,,,,,,,,,,,,,
2024-04-29,178.28,178.92,177.24,177.64,666226,48.318545,-73.177083,179.057273,178.794697,181.194545,...,-1.572959,1.833445,0.332724,-5.359616,0.158822,25.768977,17.933439,25.671439,8.677590e+06,178.794697
2024-04-26,174.94,178.06,174.02,177.62,1090481,48.271713,-73.307292,178.461515,178.598914,180.501818,...,-1.407532,1.289854,-0.168694,-4.113582,0.330811,26.148180,16.432094,30.194831,8.665312e+06,178.598914
2024-04-25,174.60,175.06,171.90,173.70,1244085,39.928140,-89.182692,177.327879,177.782428,179.203636,...,-1.502312,0.848250,-0.488238,-7.596553,0.343169,27.148450,14.536736,30.987031,5.919668e+06,177.782428
2024-04-24,174.86,176.70,173.76,174.64,1222622,42.547544,-78.526646,176.567273,177.258690,177.952727,...,-1.441639,0.260207,-0.861025,-7.303609,0.176234,27.040637,17.040569,29.053653,6.581306e+06,177.258690
2024-04-23,174.52,175.68,173.38,175.34,1100624,44.528893,-70.896785,176.131818,176.938909,177.158182,...,-1.298741,-0.333221,-1.163562,-4.747936,0.195955,27.079024,16.188746,28.427208,7.022463e+06,176.938909
2024-04-22,174.84,175.86,173.44,174.02,887529,41.556157,-76.126126,175.608788,176.452424,176.470909,...,-1.237750,-0.766513,-1.277484,-4.163454,0.166042,26.994460,15.710156,26.872484,6.354311e+06,176.452424
2024-04-19,172.92,173.36,171.82,172.96,1654861,39.242013,-85.785536,175.023636,175.870353,175.818182,...,-1.211880,-1.077700,-1.270936,-3.985789,0.102117,27.537253,14.895500,29.297442,5.346293e+06,175.870353
2024-04-18,175.50,177.08,174.32,175.52,1201705,47.072098,-47.887324,174.973939,175.811961,175.469091,...,-0.975596,-1.221345,-1.131665,-2.140946,0.045293,25.733733,22.172127,26.469875,7.124950e+06,175.811961
2024-04-17,172.98,174.76,172.30,172.68,750849,40.675271,-87.887324,174.509091,175.289968,175.020000,...,-0.962209,-1.435725,-1.076836,-2.781218,-0.024119,24.958252,20.473397,29.248189,5.910039e+06,175.289968


In [72]:
def label_data(df, window_size=11):
    results = []
    for counter_row in range(len(df)):
        if counter_row >= window_size:
            window_begin_index = counter_row - window_size
            window_end_index = window_begin_index + window_size - 1
            window_middle_index = (window_begin_index + window_end_index) // 2
            
            window_data = df['close'][window_begin_index:window_end_index + 1]
            min_index = window_data.idxmin()
            max_index = window_data.idxmax()
            
            if max_index == window_middle_index:
                results.append('SELL')
            elif min_index == window_middle_index:
                results.append('BUY')
            else:
                results.append('HOLD')
        else:
            results.append('HOLD')  # Default label for the initial period less than window size
    
    return results

df['Label'] = label_data(df)

df.sample(15)

,open,high,low,close,volume,RSI,William%R,WMA,EMA,SMA,...,PPO,PPO_signal,ROC,CMFI,DMI,DI+,DI-,PSI,TripleEMA,Label
date,,,,,,,,,,,,,,,,,,,,,
